# Capitaux 01: AZ Capitaux Processor Testing

**Purpose**: Test AZ capital extraction from IPF_AZ data

**Tests**:
1. Read IPF_AZ bronze data
2. Apply business filters
3. Extract 8 capital types (SMP, LCI, PE, RD, RC limits...)
4. Calculate capital normalization (100% basis)

---

In [ ]:
import sys
from pathlib import Path

project_root = Path.cwd().parent.parent
sys.path.insert(0, str(project_root))
print(f"Project root: {project_root}")

In [ ]:
from pyspark.sql import SparkSession
from azfr_fsspec_utils import fspath
import azfr_fsspec_abfs

azfr_fsspec_abfs.use()

spark = SparkSession.builder \
    .appName("Capitaux_AZ_Testing") \
    .getOrCreate()

print(f"✓ Spark {spark.version}")

## 1. Load Configuration

In [ ]:
from utils.loaders.config_loader import ConfigLoader
from utils.loaders.transformation_loader import TransformationLoader
import json

config = ConfigLoader(str(project_root / "config" / "config.yml"))
loader = TransformationLoader(str(project_root / "config" / "transformations"))

# Load capitaux extraction config
cap_config_path = project_root / "config" / "transformations" / "capitaux_extraction_config.json"
with open(cap_config_path, 'r') as f:
    capital_config = json.load(f)

# Remove comments
capital_types = {k: v for k, v in capital_config.items() if not k.startswith('_')}

print(f"Capital types to extract: {list(capital_types.keys())}")

## 2. Read Bronze Data

In [ ]:
from src.reader import BronzeReader

VISION = "202509"

bronze_reader = BronzeReader(
    spark, config, 
    str(project_root / "config" / "reading_config.json")
)

df = bronze_reader.read_file_group('ipf_az', VISION)
print(f"✓ Read {df.count():,} rows")

## 3. Apply Business Filters

In [ ]:
# CORRECTED: Fixed import path
from utils.transformations import apply_business_filters

business_rules = loader.get_business_rules()
az_filters = business_rules['business_filters']['az']

df_filtered = apply_business_filters(df, az_filters)
print(f"✓ After filters: {df_filtered.count():,} rows")

## 4. Extract Capitals

In [ ]:
from utils.transformations import extract_capitals

df_capitals = extract_capitals(df_filtered, capital_types)

# Show extracted capitals
capital_cols = [c for c in df_capitals.columns if any(
    x in c for x in ['smp', 'lci', 'perte', 'risque', 'limite_rc']
)]

print(f"✓ Capital columns created: {capital_cols}")
df_capitals.select('nopol', *capital_cols[:4]).show(5)

## 5. Capital Statistics

In [ ]:
from pyspark.sql.functions import sum as spark_sum, count, when, col

for cap_col in capital_cols[:4]:
    if cap_col in df_capitals.columns:
        stats = df_capitals.agg(
            count(when(col(cap_col) > 0, True)).alias('non_zero'),
            spark_sum(cap_col).alias('total')
        ).collect()[0]
        print(f"{cap_col}: {stats['non_zero']:,} non-zero, total={stats['total']:,.0f}")

## Summary

In [ ]:
print("="*60)
print("AZ CAPITAUX TESTING COMPLETE")
print("="*60)
print(f"Capital types: {list(capital_types.keys())}")
print("\n→ Next: Notebook 02 - AZEC Capitaux")